In [9]:
# https://python-binance.readthedocs.io/en/latest/overview.html
# https://www.binance.com/en/support/faq/c-6
# https://www.binance.com/en/my/settings/api-management

In [10]:
import requests
import numpy as np
import pandas as pd

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from futures_sign import send_signed_request, send_public_request
from cred import KEY, SECRET

In [11]:
symbol='ETHUSDT'
client = Client(KEY,SECRET)


In [12]:
# Get last 500 kandels 5 minutes for Symbol

def get_futures_klines(symbol,limit=500):
    x = requests.get('https://www.binance.com/fapi/v1/klines?symbol='+symbol+'&limit='+str(limit)+'&interval=5m')
    df=pd.DataFrame(x.json())
    df.columns=['open_time','open','high','low','close','volume','close_time','d1','d2','d3','d4','d5']
    df=df.drop(['d1','d2','d3','d4','d5'],axis=1)
    df['open']=df['open'].astype(float)
    df['high']=df['high'].astype(float)
    df['low']=df['low'].astype(float)
    df['close']=df['close'].astype(float)
    df['volume']=df['volume'].astype(float)
    print(df)
    return(df)

In [13]:
get_futures_klines(symbol)

         open_time     open     high      low    close     volume  \
0    1673448900000  1331.86  1332.39  1331.15  1332.04   5690.094   
1    1673449200000  1332.03  1333.53  1331.26  1332.89   6536.419   
2    1673449500000  1332.88  1334.63  1332.88  1333.67  13128.698   
3    1673449800000  1333.67  1334.33  1332.65  1332.77   6079.380   
4    1673450100000  1332.77  1333.87  1331.68  1331.80   9630.572   
..             ...      ...      ...      ...      ...        ...   
495  1673597400000  1407.85  1408.88  1406.79  1408.11   6528.491   
496  1673597700000  1408.11  1409.68  1408.10  1408.51   7831.243   
497  1673598000000  1408.50  1410.15  1408.09  1409.89   8357.496   
498  1673598300000  1409.89  1409.92  1409.12  1409.70   5237.131   
499  1673598600000  1409.70  1410.10  1408.38  1408.87   4652.111   

        close_time  
0    1673449199999  
1    1673449499999  
2    1673449799999  
3    1673450099999  
4    1673450399999  
..             ...  
495  1673597699999  
496

,open_time,open,high,low,close,volume,close_time
0,1673448900000,1331.86,1332.39,1331.15,1332.04,5690.094,1673449199999
1,1673449200000,1332.03,1333.53,1331.26,1332.89,6536.419,1673449499999
2,1673449500000,1332.88,1334.63,1332.88,1333.67,13128.698,1673449799999
3,1673449800000,1333.67,1334.33,1332.65,1332.77,6079.380,1673450099999
4,1673450100000,1332.77,1333.87,1331.68,1331.80,9630.572,1673450399999
...,...,...,...,...,...,...,...
495,1673597400000,1407.85,1408.88,1406.79,1408.11,6528.491,1673597699999
496,1673597700000,1408.11,1409.68,1408.10,1408.51,7831.243,1673597999999
497,1673598000000,1408.50,1410.15,1408.09,1409.89,8357.496,1673598299999
498,1673598300000,1409.89,1409.92,1409.12,1409.70,5237.131,1673598599999


In [14]:
# Open position for Sybol with 

def open_position(symbol,s_l,quantity_l):
    sprice=get_symbol_price(symbol)

    if(s_l=='long'):
        close_price=str(round(sprice*(1+0.01),2))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price        

                }
            ]
        }
        responce = send_signed_request('POST', '/fapi/v1/batchOrders', params)
       
    if(s_l=='short'):
        close_price=str(round(sprice*(1-0.01),2))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price
                }
           ]
        }
        responce = send_signed_request('POST', '/fapi/v1/batchOrders', params)
        

In [15]:
# Close position for symbol with quantity

def close_position(symbol,s_l,quantity_l):
    sprice=get_symbol_price(symbol)

    if(s_l=='long'):
        close_price=str(round(sprice*(1-0.01),2))
        params = {
                    "symbol":symbol,
                    "side": "SELL",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price
                }
        responce = send_signed_request('POST', '/fapi/v1/order', params)
    if(s_l=='short'):
        close_price=str(round(sprice*(1+0.01),2))
        params = {
                
                    "symbol":symbol,
                    "side": "BUY",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price        
                }
        responce = send_signed_request('POST', '/fapi/v1/order', params)



In [16]:
# Find all opened positions

def get_opened_positions(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    a = positions[positions['symbol']==symbol]['positionAmt'].astype(float).tolist()[0]
    leverage = int(positions[positions['symbol']==symbol]['leverage'])
    entryprice = positions[positions['symbol']==symbol]['entryPrice']
    profit = float(status['totalUnrealizedProfit'])
    balance = round(float(status['totalWalletBalance']),2)
    if a>0:
        pos = "long"
    elif a<0:
        pos = "short"
    else: 
        pos = ""
    return([pos,a,profit,leverage,balance,round(float(entryprice),3),0])




In [17]:
# get_opened_positions(symbol)

In [18]:
# Close all orders 

def check_and_close_orders(symbol):
    global isStop 
    a=client.futures_get_open_orders(symbol=symbol)
    if len(a)>0:
        isStop = False
        client.futures_cancel_all_open_orders(symbol=symbol)

In [19]:
def get_symbol_price(symbol):
    prices = client.get_all_tickers()
    df=pd.DataFrame(prices)
    return float(df[ df['symbol']==symbol]['price'])

In [ ]:
open_position(symbol,'long',0.01)

In [ ]:
get_opened_positions(symbol)

In [ ]:
close_position(symbol,'long',0.02)

In [ ]:
get_opened_positions(symbol)